# Using Data From Hard-Disck Memory
Here some functions that uses data from memory to create the ML model

#### TODO:
- try parallel computing (boost computational time to create datasets)
- try LightGBM (good for sparse matrixes)
- try CatBoost (avoid overfitting)
- try Bayesian optimization for parametrization tuning

In [ ]:
#  Data Manipulation
import os
import pickle
import numpy as np
import pandas as pd
pd.options.display.precision = 15
np.seterr(divide='ignore', invalid='ignore')

#  Machine learning
# from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

# Other libraries
from tqdm import tqdm
from os import listdir
from os.path import isfile, join

#  Our functions
import sys
sys.path.append("../..")
from earthquakes.helpers import (Caller,
                                 create_feature_dataset_source,
                                 create_signal_dataset,
                                 check_distributions_train_test)

from earthquakes.engineering import FeatureComputer
from earthquakes.modeling import predict_on_test
from common.utils import save_object, load_object

#  Data directories
data_dir = "../data"
save_dir = "../data/chunk_signal"
save_class = "../data/classes"
data_cycle = "../data/data cycles"
data_dp = "../data/data deep learning"

## 0. Set up the data
First, the data must be stored in memory in chunks. In order to do so, __we have to run this block of code only ones__. Unless you want to change parameters such as size. In this case, you have to run it again. Be aware that you have to delete the files on the destiny folder.

In [ ]:
# train = pd.read_csv(os.path.join(data_dir, "train.csv"),
#                     dtype={"acoustic_data": np.int16, "time_to_failure": np.float64})
train = pd.read_pickle(os.path.join(data_dir, "train.pickle"))
caller_cls = Caller(save_dir=save_dir, size=150000)
caller_cls.save_data(train)
save_object(save_path=os.path.join(save_class, "caller_cls.pkl"), object_ = caller_cls)
train = None

## 1. Lest get the events
Here a very simple implementation to illustrate how this method can be used getting the index values when there is an earthquake. Before we were having memory errors.


In [ ]:
caller_cls = load_object(os.path.join(save_class, "caller_cls.pkl"))

To ilustrate the most basic funtionality of this class, here you can set any initial id and window size. You will get back only the information that you require.

In [ ]:
caller_cls.get_intervals(i_init=60, window_size=150000)

We can iterate over all the signal to get the events (earthquakes). This is done in only two minutes! Which is remarkable considering the size of the dataset. The events positions are:
- [5656573,
 50085877,
 104677355,
 138772452,
 187641819,
 218652629,
 245829584,
 307838916,
 338276286,
 375377847,
 419368879,
 461811622,
 495800224,
 528777114,
 585568143,
 621985672]

In [ ]:
def get_events():
    """Funtion to get events (earthquakes). 
    Because there is not 0 time, an event is defined
    when the difference between two observatiosn in time
    is positive.
    """
    events = []
    for idx in tqdm(caller_cls.index_list[:-1]):
        try:
            delta = caller_cls.get_intervals(i_init = idx, window_size=150000)['time_to_failure'].diff()
        except:
            print(idx)
        delta = delta[delta > 0]
        events.append(list(delta.index -1))
        delta = None
    return [x[0] for x in events if x]

events_id = get_events()

## 2. Geting the data per cycle (information between events):
It makes sense to train and test our models on different cycles. In order to do so, here we create the datasets for every cycle independently. I skip the cycle that starts from 0 because there is no guarantee that it is a complete cycle. Same issue with the last interval. 

In [ ]:
caller_cls = load_object(os.path.join(save_class, "caller_cls.pkl"))
q = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99]
big = [100, 200, 500, 1000]
stalta = [(50, 1000), (100, 1500), (500, 5000), (1000, 10000), (5000, 15000), (10000, 25000)]
stalta_window = [(50, 1000), (100, 1500), (500, 5000), (1000, 5000)]
exp_mov_ave = [300, 3000, 10000]
exp_mov_ave_window = [300, 1000, 2000]

computer = FeatureComputer(quantiles=q, abs_quantiles=q, count_abs_big=big, stalta=stalta, stalta_window=stalta_window,
                           exp_mov_ave=exp_mov_ave, exp_mov_ave_window=exp_mov_ave_window, window=15000)
stft_computer = FeatureComputer(quantiles=q, abs_quantiles=q, count_abs_big=big) # no windows, STALTA, and exp_mov_ave for stft

events_id = [0, 5656573, 50085877, 104677355, 138772452, 187641819, 218652629, 245829584,
          307838916, 338276286, 375377847, 419368879, 461811622, 495800224, 528777114,
          585568143, 621985672]
tuples = [(x, y - 150000) for x, y in zip(events_id, events_id[1:])]

In [ ]:
for i_0, i_n in tuples:
    name = str(i_0) + '_' + str(i_n)
    new_data = None
    new_data = create_feature_dataset_source(caller_cl=caller_cls,
                                             feature_computer=computer,
                                             step=4096,
                                             stft_feature_computer=stft_computer,
                                             stft=True,
                                             window_size=150000,
                                             events_id=(i_0, i_n))
    new_data.to_pickle(os.path.join(data_cycle, "cycle_s5000_{}.pkl".format(name)))


#### Get the raw signal data for Deep learning
Here we get the data on the proper format to train deep learning algorithms. In this format every row is acoustic signal with 150000 observations before the event happens. We will get 1'600.000 ys values for deep learning.

In [ ]:
caller_cls = load_object(os.path.join(save_class, "caller_cls.pkl"))
events_id = [0, 5656573, 50085877, 104677355, 138772452, 187641819, 218652629, 245829584,
          307838916, 338276286, 375377847, 419368879, 461811622, 495800224, 528777114,
          585568143, 621985672]
number_chuncks = 40 #  Number of partitions for our dataset

index_list = np.linspace(events_id[0], events_id[-1:], number_chuncks).astype(int)
tuples = [(x, y - 150000) for x, y in zip(index_list, index_list[1:])]

In [ ]:
for i_0, i_n in tuples:
    name = str(i_0) + '_' + str(i_n)
    new_data = None
    new_data = create_signal_dataset(caller_cl=caller_cls,
                                     step=4096,
                                     window_size=150000,
                                     events_id=(i_0, i_n))
    new_data.to_pickle(os.path.join(data_dp, "signal_s5000_{}.pkl".format(name)))

## 3. Some experiments for fun

In [ ]:
onlyfiles = [f for f in listdir(data_cycle) if isfile(join(data_cycle, f))]
print('there are {} cycles to train the model'.format(len(onlyfiles)))

In [ ]:
data_train = pd.DataFrame([])
data_val = pd.DataFrame([])
n_train = 14 # 13 only one dataset to test
for i, name in enumerate(onlyfiles):
    if i <= n_train:
        data_train = pd.concat([data_train,
                                pd.read_pickle(os.path.join(data_cycle, name))])
    else:
        data_val = pd.concat([data_train,
                                pd.read_pickle(os.path.join(data_cycle, name))])


In [ ]:
X_train, X_valid, y_train, y_valid  = data_train, data_val, data_train["time_to_failure"], data_val["time_to_failure"]
del X_train["time_to_failure"]
del X_valid["time_to_failure"]

### XGBoost

In [ ]:
params_xgb = {'num_leaves': 54,
              'min_data_in_leaf': 79,
              'objective': 'huber',
              'max_depth': -1,
              'learning_rate': 0.01,
              "boosting": "gbdt",
              "bagging_freq": 5,
              "bagging_fraction": 0.8126672064208567,
              "bagging_seed": 11,
              "metric": 'mae',
              'eval_metric': 'mae',
              'reg_alpha': 0.1302650970728192,
              'reg_lambda': 0.3603427518866501,
              'num_boost_round':20000,
              'early_stopping_rounds':10,
              'tree_method':'hist'}

model = XGBRegressor(params=params_xgb,
                     verbose=False,
                     n_jobs=-1)

# model = LinearRegression(normalize=True) 

### Train the model 

In [ ]:
model.fit(X_train, y_train,
         eval_set = ((X_train, y_train), (X_valid, y_valid))) # This parametrization avoids overfit

scores = -1 * cross_val_score(model, X_valid, y_valid, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

In [ ]:
q = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.7, 0.8, 0.9, 0.95, 0.98, 0.99]
big = [100, 200, 500, 1000]
stalta = [(50, 1000), (100, 1500), (500, 5000), (1000, 10000), (5000, 15000), (10000, 25000)]
stalta_window = [(50, 1000), (100, 1500), (500, 5000), (1000, 5000)]
exp_mov_ave = [300, 3000, 10000]
exp_mov_ave_window = [300, 1000, 2000]

computer = FeatureComputer(quantiles=q, abs_quantiles=q, count_abs_big=big, stalta=stalta, stalta_window=stalta_window,
                           exp_mov_ave=exp_mov_ave, exp_mov_ave_window=exp_mov_ave_window, window=15000)
stft_computer = FeatureComputer(quantiles=q, abs_quantiles=q, count_abs_big=big) # no windows, STALTA, and exp_mov_ave for stft

submission = predict_on_test(model=model,
                             feature_computer=computer,
                             stft_feature_computer=stft_computer,
                             ycol="time_to_failure",
                             stft=True,
                             data_dir=data_dir)

In [ ]:
submission.to_csv(os.path.join(data_dir, "submissions", "submission_full_cycles_XGBoost_extened_dataset.csv"), index=False)

## 4. Compare distributions
Here we check if the data distributions of the features on the train and test are the same Kolmogorov-Smirnov test.


In [ ]:
# load featues of the test set
x_test = pickle.load(open(os.path.join(data_dir, "x_test.pickle"), "rb"))

In [ ]:
feature_names = ["all_mad", "all_0.95-quantile", "all_0.95-abs_quantile", "all_hilbert"]
feature_names = x_test.columns[0:5]
check_distributions_train_test(x_train=data_train[feature_names],
                               x_test=x_test[feature_names],
                               n_col=2,
                               alpha=0.05,
                               density=True,
                               bins=50)